# Libraries

In [5]:
import datetime
import json
import pandas as pd
import requests
import time
import random
import sys
import os
import os.path

# from mcmetadata import extract
# import mediacloud.api

from tqdm.notebook import trange  # to display a progress bar
import ipywidgets as widgets
from IPython.display import display

# Init the connection
# mc = mediacloud.api.DirectoryApi("56196a395ee77c33a296073fa08e72f541362a10")

# Dataset

In [7]:
# query = "Query_NYT_5y_surveillance"
# query = "Query_NYT_13y_surveillance"
# query = "Query_US_5y_Svalbard"
query = "wired"

CSV_FILE = query + ".csv"
FEATHER_FILE = query + ".feather"

if os.path.isfile(FEATHER_FILE):
    df = pd.read_feather(FEATHER_FILE)

else:
    df = pd.read_csv(CSV_FILE)

    df["year"] = df.loc[:, "publish_date"]
    df["year"] = df["year"].str[:4]
    df["text"] = ""
    df["text_downloaded"] = False

    # df = df.drop(
    #     [
    #         "ap_syndicated",
    #         "collect_date",
    #         "feeds",
    #         "guid",
    #         "media_id",
    #         "media_name",
    #         "media_url",
    #         "metadata",
    #         "language",
    #         "processed_stories_id",
    #         "publish_date",
    #         "story_tags",
    #         "word_count",
    #     ],
    #     axis=1,
    # )

df.sample(1)

,id,indexed_date,language,media_name,media_url,publish_date,title,url,year,text,text_downloaded
327,769e301623ce92c0790f4df1eddcbf3da410fd73c7a1ad...,2024-02-19 07:53:33.004855,en,wired.com,wired.com,2023-10-12,Artificial Intelligence Is Seeping Into All of...,https://www.wired.com/story/gadget-lab-podcast...,2023,,False


In [13]:
# Error Counter
total = widgets.IntText(description="Total", value=df.shape[0])
done = widgets.IntText(description="Done", value=0)
unavailable = widgets.IntText(description="Unavailable", value=0)
yes = widgets.IntText(description="Downloaded", value=0)
no = widgets.IntText(description="Errors", value=0)
display(total, done, unavailable, yes, no)

for index in trange(0, df.shape[0]):
    if df.iloc[index].text_downloaded == True:
        done.value += 1
        continue

    API = "https://archive.org/wayback/available"
    params = {
        "url": df.iloc[index].url,
        "timeout": "10",
        "closest": "either",
        "status_code": "200",
    }
    headers = {"accept": "application/json"}

    try:
        response = requests.get(API, params=params, headers=headers)
        snapshot = response.json().get("archived_snapshots", {}).get("closest")

        if snapshot:
            metadata = extract(url=snapshot.get("url"))
            df.at[index, "text"] = metadata["text_content"]
            df.at[index, "text_downloaded"] = True
            yes.value += 1

        else:
            unavailable.value += 1

    except Exception as e:
        no.value += 1

    # time.sleep(random.uniform(1, 20)) # This is not bad to run in the night
    # time.sleep(random.uniform(.1, 1))

    df.to_feather(FEATHER_FILE)

IntText(value=930, description='Total')

IntText(value=0, description='Done')

IntText(value=0, description='Unavailable')

IntText(value=0, description='Downloaded')

IntText(value=0, description='Errors')

  0%|          | 0/930 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
# df.to_feather(FEATHER_FILE)
df.sample(10)

,id,indexed_date,language,media_name,media_url,publish_date,title,url,year,text,text_downloaded
1,7a8a900dbb92a9f6a96d4637e1880c013806d81e24e568...,2024-09-10 11:43:07.590483,en,wired.com,wired.com,2024-09-10,Why Is AI So Bad at Generating Images of Kamal...,https://www.wired.com/story/bad-kamala-harris-...,2024,When Elon Musk shared an image showing Kamala ...,True
725,87db6aecd9ac906a5b049fe02375595de7e7f781e81d98...,2024-09-08 19:00:50.271447,en,wired.com,wired.com,2021-11-04,Search,https://www.wired.com/search/?query=Click%20He...,2021,,False
243,14ada046e2efb5d5735aff01fc12039108a8e1fce045ce...,2024-01-25 06:20:01,en,wired.com,wired.com,2024-01-23,The Biden Deepfake Robocall Is Only the Beginning,https://www.wired.com/story/biden-robocall-dee...,2024,,False
801,8076c635aa12ad7f3e5d7fdc81dcba5e455849424fd3af...,2024-08-28 03:13:18.021319,en,wired.com,wired.com,2021-07-29,China Cracks Down On Its Tech Giants. Sound Fa...,https://www.wired.com/story/china-cracks-down-...,2021,,False
874,4b86e8091c12360097dfb05433b978d017a5f15bd1ab79...,2024-09-04 01:53:27.428954,en,wired.com,wired.com,2021-03-31,"Dark Web | Latest News, Photos Videos",https://www.wired.com/tag/dark-web/,2021,,False
901,12d9cde6d28a0c447fff42f848247439be5f88ed491c44...,2024-08-20 08:34:41.161719,en,wired.com,wired.com,2021-02-13,A New Artificial Intelligence Makes Mistakes—o...,https://www.wired.com/story/new-artificial-int...,2021,,False
621,4aeb1298ea4c94af5cdf18816edff8251a4c91e87ca6af...,2024-05-06 10:23:23.247205,en,wired.com,wired.com,2022-07-12,"The Summer's Best Read Is About AI, Surveillan...",https://www.wired.com/story/lambda-david-musgr...,2022,,False
56,bed166d34c79b167e896b51e9af24779eb081e35560b3b...,2024-07-16 13:22:15.105546,en,wired.com,wired.com,2024-07-16,"Apple, Nvidia, Anthropic Used Thousands of Swi...",https://www.wired.com/story/youtube-training-d...,2024,,False
348,ab834e1e159cdf555d027f3de27a898db5b91611498713...,2024-02-20 13:45:44.444608,en,wired.com,wired.com,2023-09-25,ChatGPT Can Now Talk to You—and Look Into Your...,https://www.wired.com/story/chatgpt-can-now-ta...,2023,,False
764,2809d3a413eacd99feafaaf3f3ff78fcd11d8f232c728a...,2024-09-09 14:29:42.436025,en,wired.com,wired.com,2021-09-14,A Stanford Proposal Over AI's 'Foundations' Ig...,https://www.wired.com/story/stanford-proposal-...,2021,,False
